## print the url for each WARC file

## extract all the clean domain from the domain file

In [17]:
from urllib.parse import urlparse
import tldextract
import pandas as pd 


ext = tldextract.extract('http://forums.news.cnn.com/')

def return_domain(row):
    return tldextract.extract(row['origin_domain']).domain
df = pd.read_csv("potential_trackers",names = ['origin_domain'])

df['domain'] = df.apply(lambda x:tldextract.extract(x['origin_domain']).domain,axis = 1)
df.to_csv("./extract_trackers/domain_comparasion.csv",index = None)
df_clean_domain = pd.DataFrame({"domain":df['domain'].unique()})
df_clean_domain.to_csv("./extract_trackers/clean_potential_domain.csv",index = None)

print(df.head())


                                       origin_domain             domain
0                          00000.fls.doubleclick.net        doubleclick
1                                            0001.mn               0001
2  000337bea6c5fa91dd9c9f0407a698c5.safeframe.goo...  googlesyndication
3  00063088328760d22b34a3a1f7333979.safeframe.goo...  googlesyndication
4  000db166a3a1a46d00cbf15b933e66c1.safeframe.goo...  googlesyndication


In [1]:
from warcio.archiveiterator import ArchiveIterator
from urllib.parse import urlparse
with open('example.warc.gz', 'rb') as stream:
    for record in ArchiveIterator(stream):
        if record.rec_type == 'response':
            url = record.rec_headers.get_header('WARC-Target-URI')
            url_split = urlparse(url)
            domain = '.'.join(url_split.netloc.split('.')[-2:])
            print(domain)

archive.org


## WARC Writing

In [1]:
from warcio.capture_http import capture_http
from warcio import WARCWriter
import requests  # requests *must* be imported after capture_http

# with open('example.warc.gz', 'wb') as fh:
#     warc_writer = WARCWriter(fh)
#     with capture_http(warc_writer):
#         requests.get('https://web.archive.org/web/20171127171549/https://pypi.org/')

with open('dataset_archive/www.fujita-hu.ac.jp.gz', 'wb') as fh:
    warc_writer = WARCWriter(fh)
    with capture_http(warc_writer):
        requests.get('https://web.archive.org/web/20211010070014/https://fujita-hu.ac.jp/')


TypeError: write() argument must be str, not bytes

In [2]:
import pycountry

# print(len(pycountry.countries))

country_dict  = {country.alpha_2:country.name for country in pycountry.countries}
# for country in pycountry.countries:
#     print(country.alpha_2,country.name)

# print(country_dict)


## BeautifulSoup

In [24]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
<script type="text/javascript" src="http://www.google.com/jsapi"></script>
"""

In [25]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

In [29]:
body = soup.body

s = soup.find_all("script")

for ss in s:
    print(ss.get('src'))

l  = soup.find_all('a')
for i in l:
    print(i.get('href'))




http://www.google.com/jsapi
http://example.com/elsie
http://example.com/lacie
http://example.com/tillie


### 获得warc文件中的字段

In [5]:
filename = 'crawl-data/CC-MAIN-2021-10/segments/1614178349708.2/warc/CC-MAIN-20210224223004-20210225013004-00112.warc.gz'
offset = 203940751
length = 121563

import boto3 

from botocore import UNSIGNED
from botocore.client import Config
import json
from warcio.archiveiterator import ArchiveIterator
import gzip
# Boto3 anonymour login to common crawl
s3 = boto3.client('s3',config = Config(signature_version = UNSIGNED))
# Count the range
offset_end = offset + length - 1
byte_range = 'bytes={offset}-{end}'.format(offset=offset, end=offset_end)
gzipped_text = s3.get_object(Bucket='commoncrawl', Key=filename, Range = byte_range)['Body']

data = gzip.decompress(gzipped_text.read())
text = data.decode('utf-8')
with open("example_from_s3.warc","w") as fout:
    fout.write(text)
# print(gzipped_text.read().decode("utf-8"))

# for record in ArchiveIterator(gzipped_text):
#     print(record.rec_headers)
#     url = record.rec_headers.get_header('WARC-Target-URI')
#     text = record.content_stream().read()
#     print(url)
#     print(text)

# args = {'Range':byte_range}
# s3.download_file("commoncrawl",filename,'hello.zip',ExtraArgs = args)


## 获取wat文件

In [33]:
import json
import requests
from warcio import ArchiveIterator

warc_url = 'https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-24/segments/1590347387219.0/warc/CC-MAIN-20200525032636-20200525062636-00381.warc.gz'
wet_url = warc_url.replace('/warc/', '/wet/').replace('warc.gz', 'warc.wet.gz')
wat_url = warc_url.replace('/warc/', '/wat/').replace('warc.gz', 'warc.wat.gz')

r = requests.get(warc_url, stream=True)
records = ArchiveIterator(r.raw)
record = next(records)
record.rec_type

'warcinfo'

In [34]:
a = record.content_stream().read()
print(a.decode('utf-8'))

isPartOf: CC-MAIN-2020-24
publisher: Common Crawl
description: Wide crawl of the web for May/June 2020
operator: Common Crawl Admin (info@commoncrawl.org)
hostname: ip-10-67-67-182.ec2.internal
software: Apache Nutch 1.16 (modified, https://github.com/commoncrawl/nutch/)
robots: checked via crawler-commons 1.1-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
format: WARC File Format 1.1
conformsTo: http://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/



In [36]:
r = requests.get(wat_url, stream=True)
records = ArchiveIterator(r.raw)
a = record.content_stream().read()
data = json.loads(a.decode('utf-8'))
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)